## Cricket Statistics WebScraping using Python

In [29]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import sqlite3
from IPython.display import display,HTML

In [50]:
url='http://www.espncricinfo.com/icc-cricket-world-cup-2015/content/current/series/509587.html'

In [51]:
page=requests.get(url)

In [52]:
page.status_code

200

In [33]:
soup=BeautifulSoup(page.text,"html.parser")

In [34]:
print(soup.prettify())

<!DOCTYPE html>
<!-- hostname: web001, edition-view: espncricinfo-en-in, country: in, cluster: ind, created: 2017-12-23 18:02:33 -->
<!--[if IE 8]><html class="lt-ie9" lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml"> <![endif]-->
<!--[if IE 9]><html class="ie9" lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml"> <![endif]-->
<!--[if gt IE 9]><!-->
<html lang="en">
 <!--<![endif]-->
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <script type="text/javascript">
   var _sf_startpt=(new Date()).getTime()
  </script>
  <meta content="ZxdgH3XglRg0Bsy-Ho2RnO3EE4nRs53FloLS6fkt_nc" name="google-site-verification">
   <title>
    ICC Cricket World Cup 2015 | Cricket news, live scores, fixtures, features and statistics on ESPN Cricinfo
   </title>
   <meta content="width=device-width,initial-scale=1" name="viewport"

In [35]:
#display(HTML(page.text))

In [36]:
country_links=soup.find_all('a',href=re.compile('/icc-cricket-world-cup-2015/content/squad/'))

In [37]:
conn=sqlite3.connect('Cricket.sqlite')
cur=conn.cursor()

cur.execute('''CREATE TABLE IF NOT EXISTS Countries
            (country_id INTEGER PRIMARY KEY,country TEXT)''')

cur.execute('''CREATE TABLE IF NOT EXISTS Players
            (country_id INTEGER,player_id INTEGER UNIQUE,player TEXT)''')

cur.execute('''CREATE TABLE IF NOT EXISTS Batting_Stats
                        (player TEXT,runs INTEGER,sixes INTEGER,fours INTEGER,ducks INTEGER,fifties INTEGER,hundreds INTEGER,balls_faced INTEGER,innings INTEGER)''')
cur.execute('''CREATE TABLE IF NOT EXISTS Bowling_Stats
                        (player TEXT,bowlinnings INTEGER ,overs INTEGER ,runsgiven INTEGER,maidens INTEGER,wickets INTEGER,fourw INTEGER,fivew INTEGER)''')

In [38]:
country_links

[<a href="/icc-cricket-world-cup-2015/content/squad/814789.html" name="&amp;lpos=quicklink_Afghanistan">Afghanistan</a>,
 <a href="/icc-cricket-world-cup-2015/content/squad/819741.html" name="&amp;lpos=quicklink_Australia">Australia</a>,
 <a href="/icc-cricket-world-cup-2015/content/squad/816431.html" name="&amp;lpos=quicklink_Bangladesh">Bangladesh</a>,
 <a href="/icc-cricket-world-cup-2015/content/squad/812413.html" name="&amp;lpos=quicklink_England">England</a>,
 <a href="/icc-cricket-world-cup-2015/content/squad/817409.html" name="&amp;lpos=quicklink_India">India</a>,
 <a href="/icc-cricket-world-cup-2015/content/squad/816765.html" name="&amp;lpos=quicklink_Ireland">Ireland</a>,
 <a href="/icc-cricket-world-cup-2015/content/squad/818117.html" name="&amp;lpos=quicklink_New Zealand">New Zealand</a>,
 <a href="/icc-cricket-world-cup-2015/content/squad/817901.html" name="&amp;lpos=quicklink_Pakistan">Pakistan</a>,
 <a href="/icc-cricket-world-cup-2015/content/squad/818887.html" name="&

In [39]:
for i in country_links:
    country=i.text
    country_id=i.get('href').split('/')[-1].split('.')[0]
    cur.execute('INSERT OR IGNORE INTO Countries (country_id,country) VALUES (?,?)',(country_id,country))

In [40]:
cur.execute('SELECT country_id FROM Countries')
countryid_list=list()
for row in cur:
    countryid_list.append(row[0])

In [41]:
for cid in countryid_list:
    squad_url='http://www.espncricinfo.com/icc-cricket-world-cup-2015/content/squad/'+str(cid)+'.html'
    squad_page=requests.get(squad_url)
    soup1=BeautifulSoup(squad_page.text,"html.parser")
    player_links=soup1.findAll('a', href=re.compile('/icc-cricket-world-cup-2015/content/player/'))
    for i in player_links:
        if i.text:
            player_id=player_id=i.get('href').split('/')[-1].split('.')[0]
            player=i.text.strip()
            cur.execute('INSERT OR IGNORE INTO Players (country_id,player_id,player) VALUES (?,?,?)',(cid,player_id,player))
conn.commit()

In [42]:
cur.execute('select a.country_id,a.country,b.player_id,b.player from Countries a,Players b where a.country_id=b.country_id')
play_list=list()
for row in cur:
    play_list.append(row)
play_action=['batting','bowling']

In [54]:
year=2015
for action in play_action:
    for play in play_list:
        pid=play[2]
        country_name=play[1]
        player_name=play[3]
        stats_url='http://stats.espncricinfo.com/ci/engine/player/'+str(pid)+'.html?class=2;template=results;type='+str(action)+';view=innings;year='+str(year)
        stats_page=requests.get(stats_url)
        soup2=BeautifulSoup(stats_page.text,"html.parser")
        stats=soup2.findAll(text='filtered')[0].findParents('tr')[0].findAll("td")
        if action=="bowling":
            check=[3,4,6,5,7,12,13]
        else:
            check=[4,13,12,11,10,9,7,2]
        results=[]
        for i in check:
            try:
                results.append(float(stats[i].get_text()))
            except:
                results.append(0)
        if action=="batting":
            cur.execute('INSERT OR IGNORE INTO Batting_Stats (player,runs,sixes,fours,ducks,fifties,hundreds,balls_faced,innings)VALUES(?,?,?,?,?,?,?,?,?)',(player_name,results[0],results[1],results[2],results[3],results[4],results[5],results[6],results[7]))
        elif action=="bowling":
            cur.execute('INSERT OR IGNORE INTO Bowling_Stats (player,bowlinnings,overs,runsgiven,maidens,wickets,fourw,fivew)VALUES(?,?,?,?,?,?,?,?)',(player_name,results[0],results[1],results[2],results[3],results[4],results[5],results[6]))
conn.commit()

<td class="left">filtered</td>
<td>4</td>
<td>4</td>
<td>0</td>
<td>36</td>
<td class="padAst">10</td>
<td>9.00</td>
<td>71</td>
<td>50.70</td>
<td>0</td>
<td>0</td>
<td>0</td>
<td>3</td>
<td>0</td>
<td></td>


In [55]:
conn.commit()